In [1]:
import pandas as pd
import numpy as np

In [2]:
# Open crime data bases and keep first=degree homicides
crimes = pd.read_csv('data/IDM_NM_ene2020.csv', encoding = 'latin')
crimes.columns = map(str.lower, crimes.columns)

crimes.rename(columns={'año': 'year', 
                      'clave_ent': 'state_key',
                      'entidad': 'state',
                      'cve. municipio': 'key',
                      'municipio': 'municipality',
                      'bien jurídico afectado': 'affected_good',
                      'tipo de delito': 'crime_type',
                      'subtipo de delito': 'crime_subtype',
                      'modalidad': 'modality',
                      'enero': 'January',
                      'febrero': 'February',
                      'marzo' : 'March',
                      'abril': 'April',
                      'mayo': 'May',
                      'junio': 'June',
                      'julio': 'July',
                      'agosto': 'August',
                      'septiembre': 'September',
                      'octubre': 'October',
                      'noviembre': 'November',
                      'diciembre': 'December'}, inplace=True)

crimes = crimes[(crimes.crime_subtype == 'Homicidio doloso') & (crimes.year < 2020)]

crimes.drop(columns=['state_key', 'state_key', 'affected_good', 'crime_type', 'crime_subtype', 'modality'],
           inplace=True)


In [3]:
# Open old (2011-2014) crime data bases and keep first-degree homicides

crimes_old = pd.read_csv('data/IDM_oct19.csv', encoding = 'latin')
crimes_old.columns = map(str.lower, crimes_old.columns)
crimes_old = crimes_old[(crimes_old.año < 2015) & (crimes_old.modalidad == 'HOMICIDIOS') & (crimes_old.tipo == 'DOLOSOS')]

crimes_old.rename(columns={'año': 'year', 
                      'inegi': 'key',
                      'municipio': 'municipality',
                      'enero': 'January',
                      'febrero': 'February',
                      'marzo' : 'March',
                      'abril': 'April',
                      'mayo': 'May',
                      'junio': 'June',
                      'julio': 'July',
                      'agosto': 'August',
                      'septiembre': 'September',
                      'octubre': 'October',
                      'noviembre': 'November',
                      'diciembre': 'December'}, inplace=True)

crimes_old.drop(columns=['entidad', 'modalidad', 'tipo', 'subtipo', 'municipality'], inplace=True)

crimes_old=crimes_old.merge(crimes[['key', 'municipality', 'state']].drop_duplicates(), how='left', on='key')

In [4]:
# Append databases
crimes = crimes.append(crimes_old, sort=False)

# Aggregate homicides by year

crimes = crimes.melt(id_vars=['key', 'municipality', 'state', 'year'],
                   value_vars=['January', 'February', 'March', 'April', 'May', 'June',
                               'July', 'August', 'September', 'October', 'November', 'December'],
                   var_name='month', value_name='homicides')

crimes.fillna(0, inplace=True)
crimes = crimes.groupby(['key', 'municipality', 'state', 'year'], as_index=False).sum()

In [5]:
# Create columns per year
crimes = crimes.pivot_table(values='homicides', index=['key', 'municipality', 'state'], columns='year',
                            fill_value=0).reset_index()


In [6]:
# Open population estimates
pop = pd.read_csv('data/base_municipios_final_datos_01-1.csv', encoding = 'latin')
pop2 = pd.read_csv('data/base_municipios_final_datos_02-1.csv', encoding = 'latin')
pop = pop.append(pop2)

pop.columns = map(str.lower, pop.columns)

pop.rename(columns={'año': 'year', 
                    'clave': 'key',
                    }, inplace=True)

pop = pop[pop.year < 2020]

pop.drop(columns=['renglon', 'clave_ent', 'nom_ent', 'mun', 'sexo', 'edad_quin'], inplace=True)

pop = pop.groupby(['key', 'year'], as_index=False).sum()


In [7]:
# pivot population
pop['year'] = 'pop' + pop.year.apply(str)
pop = pop.pivot_table(values='pob', index='key', columns='year',
                      fill_value=0).reset_index()

In [8]:
# Get file of 2010 population Source: INEGI. Censo de Poblacon y Vivienda 2010.
pop3 = pd.read_csv('data/INEGI_Exporta_20200229165754.csv', encoding = 'latin')
pop3.key.replace('\\s', value='', regex=True, inplace=True)
pop3['key'] = pop3.key.astype(np.int64)
pop3.rename(columns={'Total': 'pop2010'}, inplace=True)
pop3.pop2010.replace('\\,', value='', regex=True, inplace=True)
pop3['pop2010'] = pop3.pop2010.astype(np.int64)
pop3.drop(columns=['mun', 'Hombre', 'Mujer'], inplace=True)
pop3 = pop3[pop3.key > 1000]

In [9]:
# Merge population data
pop = pop.merge(pop3, how='inner', on='key')

In [10]:
# Merge homicide and population data
homicides = crimes.merge(pop, how='inner', on='key')

In [11]:
# Create homicide rate variables

for y in range(2011, 2020):
    if y < 2015:
        pop = 'pop2010'

    else:
        pop = 'pop' + str(y)

    var = 'homrate' + str(y)

    homicides[var] = homicides[y] / (homicides[pop]/100000)

In [12]:
# Create region variable
homicides.loc[homicides.state == 'Coahuila de Zaragoza', 'state'] = 'Coahuila'
homicides.loc[homicides.state == 'Michoacán de Ocampo', 'state'] = 'Michoacán'
homicides.loc[homicides.state == 'Veracruz de Ignacio de la Llave', 'state'] = 'Veracruz'

nw = ['Baja California', 'Baja California Sur', 'Chihuahua', 'Sinaloa', 'Sonora']
ne = ['Coahuila', 'Durango', 'Nuevo León', 'San Luis Potosí', 'Tamaulipas']
w = ['Aguascalientes', 'Colima', 'Guanajuato', 'Jalisco', 'Michoacán', 'Nayarit', 'Querétaro', 'Zacatecas']
c = ['Ciudad de México', 'México', 'Guerrero', 'Hidalgo', 'Morelos', 'Puebla', 'Tlaxcala']
se = ['Campeche', 'Chiapas', 'Oaxaca', 'Quintana Roo', 'Tabasco', 'Veracruz', 'Yucatán']

homicides['region'] = np.where(homicides.state.isin(nw), 'Northwest',
                                np.where(homicides.state.isin(ne), 'Northeast',
                                         np.where(homicides.state.isin(w), 'West', 
                                                  np.where(homicides.state.isin(c), 'Central', 'Southeast'))))


In [13]:
# Replace Nan
homicides.fillna(0, inplace=True)

#write file
homicides.to_csv('data/homicides.csv')

In [25]:
homregion = homicides.groupby(['region'], as_index=False).sum()

In [26]:
for y in range(2011, 2020):
    if y < 2015:
        pop = 'pop2010'

    else:
        pop = 'pop' + str(y)

    var = 'homrate' + str(y)

    homregion[var] = homregion[y] / (homregion[pop]/100000)

In [27]:
homregion

,region,key,2011,2012,2013,2014,2015,2016,2017,2018,...,pop2010,homrate2011,homrate2012,homrate2013,homrate2014,homrate2015,homrate2016,homrate2017,homrate2018,homrate2019
0,Central,10981962,5692,6864,5965,5213,6014,6551,7163,8063,...,38807720,14.667185,17.687202,15.370653,13.432894,14.495538,15.648423,16.964247,18.940867,18.577804
1,Northeast,4150504,4550,3946,2657,2052,1738,1989,2303,2463,...,14888855,30.559771,26.503045,17.845563,13.782121,10.701632,12.092811,13.835528,14.626787,13.988223
2,Northwest,2888115,5564,4177,3706,3284,3347,4167,6237,6473,...,12628802,44.058019,33.075188,29.345618,26.004050,24.619952,30.279480,44.723207,45.825715,47.095369
3,Southeast,22780293,1780,1780,1810,1470,1703,2640,3564,4374,...,22583935,7.881709,7.881709,8.014547,6.509052,6.943691,10.633045,14.192369,17.227937,16.313570
4,West,6764610,3086,3200,2933,2965,3230,4460,5429,7665,...,23427226,13.172708,13.659321,12.519621,12.656215,12.687103,17.304738,20.822591,29.075398,30.261849


In [48]:
homicides.sort_values(by=2013, ascending=False).head(20)

,key,municipality,state,2011,2012,2013,2014,2015,2016,2017,...,homrate2011,homrate2012,homrate2013,homrate2014,homrate2015,homrate2016,homrate2017,homrate2018,homrate2019,region
366,12001,Acapulco de Juárez,Guerrero,1008,1170,883,590,902,918,833,...,127.599621,148.106703,111.776255,74.686286,109.497912,110.844264,100.162087,100.372624,70.651642,Central
14,2004,Tijuana,Baja California,418,332,492,462,608,865,1613,...,26.800318,21.286377,31.544872,29.621404,36.686066,51.506521,94.427656,129.140101,113.430320,Northwest
234,8037,Juárez,Chihuahua,1460,647,453,388,269,470,636,...,109.598831,48.568797,34.005665,29.126265,18.996169,32.984354,44.289108,69.902268,87.985422,Northwest
1875,25006,Culiacán,Sinaloa,527,484,420,358,395,381,575,...,61.376273,56.368341,48.914676,41.693939,42.975937,40.968623,61.241487,52.779083,47.627023,Northwest
688,15033,Ecatepec de Morelos,México,325,383,312,349,349,298,318,...,19.624336,23.126525,18.839362,21.073518,20.840810,17.736902,18.859823,18.669017,17.473291,Central
985,19039,Monterrey,Nuevo León,700,551,266,124,130,207,140,...,61.644137,48.522742,23.424772,10.919819,11.942437,18.915614,12.728002,19.605770,19.730181,Northeast
216,8019,Chihuahua,Chihuahua,554,363,251,184,142,275,360,...,67.598649,44.292978,30.626825,22.451537,16.048661,30.652315,39.512807,29.206397,34.463148,Northwest
66,5035,Torreón,Coahuila,455,466,239,157,112,76,90,...,71.134986,72.854733,37.365410,24.545479,16.151757,10.807429,12.612178,11.327782,12.123639,Northeast
650,14120,Zapopan,Jalisco,250,202,217,144,170,156,208,...,20.100406,16.241128,17.447152,11.577834,12.527616,11.356811,14.965062,15.371302,17.891381,West
833,16053,Morelia,Michoacán,151,138,199,182,148,136,123,...,20.705382,18.922799,27.287225,24.956155,18.733513,17.035562,15.264239,22.514127,32.830462,West


In [61]:
homicides[homicides.pop2019 >= 100000].sort_values(by='homrate2019', ascending=False).head(30)

,key,municipality,state,2011,2012,2013,2014,2015,2016,2017,...,homrate2011,homrate2012,homrate2013,homrate2014,homrate2015,homrate2016,homrate2017,homrate2018,homrate2019,region
15,2005,Playas de Rosarito,Baja California,40,30,50,45,48,78,108,...,44.116998,33.087749,55.146248,49.631623,48.994090,78.187650,105.975861,98.104279,134.960408,Northwest
76,6007,Manzanillo,Colima,22,44,54,22,55,150,220,...,13.629042,27.258085,33.453104,13.629042,29.932299,79.824599,114.594077,108.215717,132.204256,West
14,2004,Tijuana,Baja California,418,332,492,462,608,865,1613,...,26.800318,21.286377,31.544872,29.621404,36.686066,51.506521,94.427656,129.140101,113.430320,Northwest
78,6009,Tecomán,Colima,37,92,45,22,38,159,223,...,32.822951,81.613825,39.919806,19.516349,30.538768,125.684745,173.440976,146.224573,103.287872,West
346,11027,Salamanca,Guanajuato,26,45,25,39,56,64,82,...,9.971925,17.259101,9.588390,14.957888,19.987579,22.616200,28.711987,97.857884,101.810227,West
321,11002,Acámbaro,Guanajuato,28,25,22,33,24,40,27,...,25.681005,22.929469,20.177933,30.266899,20.708757,34.375779,23.101407,66.396541,99.854449,West
403,12038,Zihuatanejo de Azueta,Guerrero,114,80,65,35,43,96,128,...,96.437726,67.675597,54.986423,29.608074,34.083973,75.347895,99.648893,108.995617,94.514327,Central
234,8037,Juárez,Chihuahua,1460,647,453,388,269,470,636,...,109.598831,48.568797,34.005665,29.126265,18.996169,32.984354,44.289108,69.902268,87.985422,Northwest
1810,23008,Solidaridad,Quintana Roo,25,18,13,21,20,26,49,...,15.692675,11.298726,8.160191,13.181847,9.860815,12.325897,22.389151,48.600955,80.997339,Southeast
361,11042,Valle de Santiago,Guanajuato,9,11,10,15,14,14,18,...,6.380354,7.798211,7.089282,10.633924,9.475722,9.410499,12.018509,60.999867,80.961000,West
